<a href="https://colab.research.google.com/github/marcusborela/Aprendizado-Profundo-Unicamp/blob/main/ExportaImagensSentinel2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Criando amostras de imagens do Sentinel 2

Remis Balaniuk, PhD

Este script é parte do curso de Sensoriamento Remoto e Deep Learning.

Seu objetivo é a automatização do procedimento de seleção, recorte e exportação de imagens processadas da constelação Sentinel 2 utilizando a API do Google Earth Engine.

In [1]:
!pip install earthengine-api
!pip install geopandas
import os
import sys
import math

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.0 MB 4.9 MB/s 
     |████████████████████████████████| 6.3 MB 39.4 MB/s 
     |████████████████████████████████| 16.7 MB 1.5 MB/s 


O usuário deve possuir uma conta Google (gmail) e ter se habilitado junto ao Google Earth Engine (see https://earthengine.google.com/).

In [2]:
# Import the Earth Engine library.
import ee

# Trigger the authentication flow.
ee.Authenticate()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=hWa9yovtkTMhVldoR3d13jKg73Cnw9bsWSVnZK6OFcE&tc=gI5mU9iNxFFuNdkJxWLfPOz-yfWFW28bzHYHBdnLaTY&cc=yEKpF5ILdLvNG6w8h8CsNZaMIks9Y-P9XnbXFKku5PU

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AX4XfWjCNBcEd6sOHWnt-O7qJAE7sA6MtQ72f4r7rQYc5WT-H9Esh70k5zw

Successfully saved authorization token.


As amostras de imagem serão salvas no Google Drive do usuário. A unidade deve ser montada antes de prosseguir.

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# Importando o pacote Python do Earth Engine
import ee
import pandas as pd
import numpy as np
import geopandas as gpd

In [5]:
# Inicializando o objeto Earth Engine usando as credenciais de autenticação.
ee.Initialize()

In [6]:
# Função de Cloud masking no Sentinel-2.
def maskS2clouds(image):
  cloudShadowBitMask = ee.Number(2).pow(3).int()
  cloudsBitMask = ee.Number(2).pow(5).int()
  qa = image.select('QA60')
  mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0).And(
    qa.bitwiseAnd(cloudsBitMask).eq(0))
  return image.updateMask(mask).select(bands).divide(10000)


Editando a próxima célula o usuário pode selecionar as bandas espectrais a serem incluídas nos patches da imagem.

In [7]:
# Use essas bandas para previsão.
#bands = ['B1', 'B2', 'B3', 'B4', 'B5','B6', 'B7', 'B8', 'B9', 'B10', 'B11', 'B12']

bands = ['B2', 'B3', 'B4', 'B5', 'B7', 'B8', 'B11']

# Use os dados de refletância de superfície do Sentinel 2.
sentinel = ee.ImageCollection("COPERNICUS/S2")


Editando a próxima célula o usuário pode selecionar o intervalo de tempo (filterDate) e a porcentagem de cobertura de nuvens ('CLOUDY_PIXEL_PERCENTAGE') para filtrar as imagens utilizadas na composição dos patches. Quanto menor o intervalo, maiores as chances de ter pixels sem dados para exibir. Regiões com cobertura de nuvens frequente, como a floresta tropical, exigirão um longo intervalo para garantir um conjunto completo de pixels.

In [25]:
# Os dados de entrada de imagem são compostos medianos mascarados em nuvem.
# imageantes = sentinel.filterDate('2019-07-01','2019-12-01').filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE', 20)).map(maskS2clouds).median().toFloat()
# imagedepois = sentinel.filterDate('2021-07-01','2021-12-01').filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE', 20)).map(maskS2clouds).median().toFloat()

# empilhando imagens
#image = ee.Image.cat(imageantes,imagedepois)


# Lixão
image = sentinel.filterDate('2021-07-01','2022-06-01').filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE', 20)).map(maskS2clouds).median().toFloat()




A seguir, o usuário poderá escolher um arquivo csv de sua raiz do Google Drive contendo as coordenadas (latitude e longitude) dos pontos dos quais deseja extrair os patches da imagem. Além disso, ele será solicitado a informar o separador de colunas utilizado no arquivo csv.

Os polígonos que delimitam as áreas de interesse descritas no arquivo csv podem ser definidos usando um dos seguintes esquemas:

1: usando dois pares de coordenadas indicando os cantos inferior esquerdo (sudoeste) e superior direito (nordeste) do polígono;

2: definir as coordenadas de um ponto central e o comprimento do lado de um quadrado definido em torno desse ponto.

O usuário será solicitado a informar qual esquema deve ser utilizado para ler o arquivo csv (todos os registros do arquivo devem utilizar o mesmo esquema).

Uma última coluna no arquivo csv deve ser usada para informar o nome da classe para a amostra. Este nome de classe será usado como prefixo para nomear os arquivos de imagem.

Os registros csv devem ficar assim:

####-separador de colunas =';' e esquema 1:

> latitude do canto inferior esquerdo; longitude do canto inferior esquerdo; latitude do canto superior direito; longitude do canto superior direito;  classe

> -20.893706;-45.271998;-18.854222;-41.958905;mina


####-separador de colunas =';' e esquema 2:
> latitude do ponto central; longitude do ponto central>; nome da classe

>-23.82113889;-50.42022222;mina



In [9]:
def offset(lat,lon,x,y):

	#raio da terra, esfera
	R=6378137

	# offsets das coordenadas calculados em radianos
	# deslocamento angular l = theta*r so theta = l/r (radians)
	#  https://www.google.com/search?q=how+to+compute+displacements+in+radians&rlz=1C1GCEA_enBR896BR896&oq=how+to+compute+displacements+in+radians&aqs=chrome..69i57j0i22i30l2.17447j0j9&sourceid=chrome&ie=UTF-8#kpvalbx=_XEeGYvSCPZed5OUPkeaP8AQ32
	dLat = x/R
	dLon = y/(R*math.cos(math.pi*lat/180))

	return lat + dLat * 180/math.pi, lon + dLon * 180/math.pi
 

def exportImage(data,scheme,size=0,labelbase = None):

	# carrega o arquivo csv

	for d in range(len(data)):

		if scheme == 2:	
			x = data.values[d][0]
			y = data.values[d][1]

			llx , lly = offset(x,y,-size/2,-size/2)
			urx , ury = offset(x,y,size/2,	size/2)

			#if label is None:
			label = labelbase + str(data.loc[d,data.columns[-1]])
	 
		else:

			llx = data.values[d][0]
			lly = data.values[d][1]
			urx = data.values[d][2]
			ury = data.values[d][3]	

			#if label is None:
			label = labelbase + str(data.loc[d,data.columns[-1]])	

		geometry = [[llx,lly], [llx,ury], [urx,ury], [urx,lly]]

		task_config = {
	    'scale':  10 ,
	    'region': geometry
	    }
		
		#print(label,d,geometry)
		name = label + str(d)
		# Create a task.
		task = ee.batch.Export.image(image, name, task_config)

		# Send the task to the earth engine.
		task.start() 
		

In [10]:
path_ref = './curso_analise_imagens_satelite/FinalLixoes/'

In [15]:
%cd "content/drive/curso_analise_imagens_satelite/FinalLixoes/"

[Errno 2] No such file or directory: 'content/drive/curso_analise_imagens_satelite/FinalLixoes/'
/content


In [18]:
 %cd /content/drive/My Drive/curso_analise_imagens_satelite/FinalLixoes/

/content/drive/My Drive/curso_analise_imagens_satelite/FinalLixoes


In [31]:
%ls


'coordenadas lixoes.csv'   coordenadas_nao_lixoes.csv


In [20]:
data = pd.read_csv('./Data/coordenadas_nao_lixoes.csv', sep= ';')

In [21]:
data[:3]

,X,Y,Id
0,-51.310442,-7.457399,nao_lixao
1,-55.397888,-12.524384,nao_lixao
2,-64.140542,-4.009832,nao_lixao


In [38]:
data = pd.read_csv('coordenadas_lixoes.csv', sep= ';')

In [39]:
data[:3]

,X,Y,id
0,-37.656223,-7.732032,lixao
1,-35.962344,-7.040988,lixao
2,-53.480445,-24.384909,lixao


In [23]:
%cd Data

/content/drive/MyDrive/curso_analise_imagens_satelite/FinalLixoes/Data


In [40]:
#MAIN WORKFLOW

# aassumindo o arquivo csv na pasta raiz Minha unidade (altere o %cd se não for o caso)
#%cd /content/drive/My Drive/
files = []
count=0
for f in os.listdir('./'):
  name, ext = os.path.splitext(f)
  if ext == '.csv':
    files.append(f)
    count+=1
    print(count,":",f)

print("Choose your file:")
try:
  r=int(input('Input:'))
except ValueError:
  print("Not a number")

print("csv separator? (typically ';' or ',')")
sep=input('Input:')

data = pd.read_csv(files[r-1], sep= sep)# .sample(200) # .reset_index(drop=True)
name, ext = os.path.splitext(files[r-1])

print(len(data),"records with",len(data.columns),"columns")

print(name,data[:3])

if len(data.columns)==3:
  print("Central point scheme. Please inform the square side length (in meters):")
  try:
    size=int(input('Input:'))
  except ValueError:
    print("Not a number")
  exportImage(data,2,size,name)
elif data.shape[1]==5:
  exportImage(data,1,name)
else:
  print("Invalid csv file!")
  sys.exit(0)

1 : coordenadas_nao_lixoes.csv
2 : coordenadas_lixoes.csv
Choose your file:
Input:2
csv separator? (typically ';' or ',')
Input:;
270 records with 3 columns
coordenadas_lixoes            X          Y     id
0 -37.656223  -7.732032  lixao
1 -35.962344  -7.040988  lixao
2 -53.480445 -24.384909  lixao
Central point scheme. Please inform the square side length (in meters):
Input:400


Se o script foi bem-sucedido, as tarefas devem estar visíveis na interface do editor de código do Google Earth Engine (https://code.earthengine.google.com/). O usuário deve efetuar logon para autorizar a execução das tarefas.

In [44]:
%cd ..

/content/drive/MyDrive


In [45]:
%ls coordenadas_nao_lixoes*.*

coordenadas_nao_lixoesnao_lixao0.tif    coordenadas_nao_lixoesnao_lixao20.tif
coordenadas_nao_lixoesnao_lixao100.tif  coordenadas_nao_lixoesnao_lixao21.tif
coordenadas_nao_lixoesnao_lixao101.tif  coordenadas_nao_lixoesnao_lixao22.tif
coordenadas_nao_lixoesnao_lixao102.tif  coordenadas_nao_lixoesnao_lixao23.tif
coordenadas_nao_lixoesnao_lixao103.tif  coordenadas_nao_lixoesnao_lixao24.tif
coordenadas_nao_lixoesnao_lixao104.tif  coordenadas_nao_lixoesnao_lixao25.tif
coordenadas_nao_lixoesnao_lixao105.tif  coordenadas_nao_lixoesnao_lixao26.tif
coordenadas_nao_lixoesnao_lixao106.tif  coordenadas_nao_lixoesnao_lixao27.tif
coordenadas_nao_lixoesnao_lixao107.tif  coordenadas_nao_lixoesnao_lixao28.tif
coordenadas_nao_lixoesnao_lixao108.tif  coordenadas_nao_lixoesnao_lixao29.tif
coordenadas_nao_lixoesnao_lixao109.tif  coordenadas_nao_lixoesnao_lixao2.tif
coordenadas_nao_lixoesnao_lixao10.tif   coordenadas_nao_lixoesnao_lixao30.tif
coordenadas_nao_lixoesnao_lixao110.tif  coordenadas_nao_lixoesnao

In [46]:
%mv coordenadas_nao_lixoes*.tif curso_analise_imagens_satelite/FinalLixoes/Data/train/naolixao

In [47]:
%mv coordenadas_lixoes*.tif curso_analise_imagens_satelite/FinalLixoes/Data/train/lixao